In [ ]:
import pandas as pd
import numpy as np
import os
import xgboost as xgb
#import keras as keras

In [ ]:
XYDataFileName = "42p_82894samples_02-23-24_incidence_XY.txt";
dataFolderPath = "/Users/adsriram98/Documents/PARK_LAB/PRS_calc/data";
mypara = {'num_epochs': 50, 'batch_size': 20}
'''
Run MGM
Note: MGM was implemented in Java and the following Python APIs call the Java implementation.
Please restart the Python program after encountering a JVM problem.
The format of the input data file must be ".txt" in which columns are separated by "\t" and it should also include the response variables.
Here is what the input data should look like:
X1 X2 ... Xp
1  1  ... 1
'''
# import the MGM package
#from MGM.MGM import MGM
# Initialize a MGM object
#mgm = MGM();
'''
Run MGM
Parameters:
    dataFolderPath: the directory that stores the input data.
    DataFileName: the name of the input data.
    lambda_continuous_continuous: the panalty value 'lambda' set for the associations whose two end variables are continuous.
    lamda_continuous_discrete: the panalty value 'lambda' set for the associations whose one end variable is continuous and the other is discrete.
    lamda_discrete_discrete: the panalty value 'lambda' set for the associations whose two end variables are discrete.
    
Return:
    mgmOutputFile: a tuple, where the first file contains all the selected associations and the second file contains the corresponding likelihoods.
'''
#mgmOutputFile = mgm.runMGM(dataFolderPath, XYDataFileName,lambda_continuous_continuous = 0.3, lamda_continuous_discrete = 0.3, lamda_discrete_discrete = 0.3);
"""
MGM uses the Python package Jpype to call MGM's Java implementation.
According to Jpype documents, it says "Due to limitations in the JPype, 
it is not possible to restart the JVM after being terminated."
Therefore, please restart the Python kernel if you encounter an OSError (i.e., "OSError: JVM cannot be restarted").
"""
#print("MGM's output was saved as the following file:");
#print(mgmOutputFile[0]);
#print("The likelihood values were saved as the following file:");
#print(mgmOutputFile[1]);

In [ ]:
'''
#Generate knockoff data using one of two methods: ISEE Omega and Cholesky_LU.
The code for generating ISEE Omega knockoff is implemented using R. Please make sure your computer has R installed.
'''
from DL.knockoff.KnockoffGenerator import KnockoffGenerator;
generator = KnockoffGenerator();

dataFolderPath = "/Users/adsriram98/Documents/PARK_LAB/PRS_calc/data";
DataFileName = "42p_82894samples_02-23-24_incidence_X.txt"
knockoffFilePath = generator.CholLuKnockoff(dataFolderPath, DataFileName,sep="\t");

# #If want to generate ISEE Omega knockoff, please set the ISEE code path and R home environment.
# generator.set_ISEE_path("/absolute_path_of_DAG_DeepVASE");#/home/user/DAG_DeepVASE/
# generator.set_R_home('absolute_path_to_directory_where_r_is_installed');#e.g.,/home/user/lib/R
#knockoffFilePath = generator.ISEEKnockoff(dataFolderPath, DataFileName,sep="\t");

print("The newly generated knockoff file is named as:");
print(knockoffFilePath);

In [ ]:
XKnockoffData = pd.read_csv(knockoffFilePath,sep="\t"); #for stat knockoffs
#XKnockoffData = pd.read_csv("/Users/adsriram98/Documents/PARK_LAB/PRS_calc/data/02-22-24-PRSknockoffs-incidence.csv", sep=",") #for PRS knockoffs

In [ ]:
''''''
# After generating the knockoff data, run DNN

dataFolderPath = "/Users/adsriram98/Documents/PARK_LAB/PRS_calc/data"
XKnockoffData = pd.read_csv(knockoffFilePath,sep="\t"); #for stat knockoffs
#XKnockoffData = pd.read_csv("/Users/adsriram98/Documents/PARK_LAB/PRS_calc/data/06-07-24-onlyIncidenceAHD_PRSknockoffs.csv", sep=",") #for PRS knockoffs

YDataFileName = "42p_82894samples_02-23-24_incidence_Y.txt";
Ydata = pd.read_csv(dataFolderPath+os.path.sep+YDataFileName,sep="\t");

XKValues = XKnockoffData.values;
YValues = Ydata.values;
    
pNum = int(XKValues.shape[1] / 2);
n = XKValues.shape[0];
print(XKValues.shape);
print(YValues.shape);
print(pNum);
    
XOrigin = XKValues[:, 0:pNum];
knockoff = XKValues[:, pNum:];

X3DTrain = np.zeros((n, pNum, 2));
X3DTrain[:, :, 0] = XOrigin;
X3DTrain[:, :, 1] = knockoff;
labelTrain = YValues;
    
coeff = 0.05 * np.sqrt(2.0 * np.log(pNum) / n);

nOutputs = Ydata.shape[1];

#Save the DNN output to the following directory.
resultDir = dataFolderPath+os.path.sep+'02_23_predictor_STATKO_DNN_result/'; #or PRSKO; change the knockoff naming and directory accordingly
if not os.path.exists(resultDir):
    os.makedirs(resultDir);
    
from DL.DNN.DNN import DNN;
#from sklearn.model_selection import cross_val_score;
#import tensorflow as tf;

paralabel = "epo" + str(mypara['num_epochs']) + "batch" + str(mypara['batch_size']);
print(paralabel);

dnn = DNN(output_layer_activation='sigmoid', output_loss = 'binary_crossentropy',
          num_epochs = mypara['num_epochs'],
          batch_size = mypara['batch_size']);

In [ ]:
X3DTrain.shape

In [ ]:
X3DTrain.shape
labelTrain.shape

In [ ]:
#labelTrain
XKValues

In [ ]:
#import tensorflow as tf;
#scores= cross_val_score(model, X3DTrain, labelTrain, cv=5)
Ydata.shape

In [ ]:
if not os.path.exists(dataFolderPath +os.path.sep+paralabel):
        os.makedirs(dataFolderPath +os.path.sep+paralabel)
        print(f"Directory '{dataFolderPath +os.path.sep+paralabel}' created successfully.")

In [ ]:

#yhat3DTrain=model.predict(X3DTrain)
#pd.DataFrame(yhat3DTrain).to_csv(dataFolderPath +os.path.sep+paralabel+os.path.sep+"test.csv")

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve, auc, confusion_matrix, accuracy_score, recall_score, precision_score, f1_score, matthews_corrcoef, precision_recall_curve, average_precision_score
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as imbalanced_pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.calibration import CalibratedClassifierCV

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(labelTrain), y=labelTrain.ravel())
class_weights = dict(enumerate(class_weights))


def load_data():
    data = X3DTrain
    labels = labelTrain
    header_info = "Your header information here"
    return data, labels, header_info

def create_model():
    dnn = DNN(output_layer_activation='sigmoid', output_loss='binary_crossentropy',
              num_epochs=mypara['num_epochs'], batch_size=mypara['batch_size'])
    model = dnn.build_DNN(pNum, nOutputs, coeff)
    callback = DNN.Job_finish_Callback(resultDir, pNum)
    return model, callback

def calculate_optimal_threshold(y_true, y_probs):
    fpr, tpr, thresholds = roc_curve(y_true, y_probs)
    optimal_idx = np.argmax(tpr - fpr)
    return thresholds[optimal_idx]

def train_and_evaluate_xgboost(data_train, labels_train, data_test, labels_test, metrics_vector_xgb):
    data_train_flattened = data_train.reshape(data_train.shape[0], -1)
    data_test_flattened = data_test.reshape(data_test.shape[0], -1)

    dtrain = xgb.DMatrix(data_train_flattened, label=labels_train)
    dtest = xgb.DMatrix(data_test_flattened, label=labels_test)

    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'scale_pos_weight': class_weights[1] / class_weights[0]
    }
    num_rounds = 100

    bst = xgb.train(params, dtrain, num_rounds)
    preds_prob = bst.predict(dtest)
    auc_score = roc_auc_score(labels_test, preds_prob)
    precision, recall, _ = precision_recall_curve(labels_test, preds_prob)
    pr_auc = auc(recall, precision)
    optimal_threshold = calculate_optimal_threshold(labels_test, preds_prob)
    mcc = matthews_corrcoef(labels_test, (preds_prob > optimal_threshold).astype(int))

    metrics_vector_xgb.append([auc_score, pr_auc, mcc])

def train_and_evaluate_rf(data_train, labels_train, data_test, labels_test, metrics_vector_rf):
    data_train_flattened = data_train.reshape(data_train.shape[0], -1)
    data_test_flattened = data_test.reshape(data_test.shape[0], -1)

    rf_model = RandomForestClassifier(class_weight='balanced', n_estimators=100)
    rf_model.fit(data_train_flattened, labels_train.ravel())
    preds_prob = rf_model.predict_proba(data_test_flattened)[:, 1]
    auc_score = roc_auc_score(labels_test, preds_prob)
    precision, recall, _ = precision_recall_curve(labels_test, preds_prob)
    pr_auc = auc(recall, precision)
    optimal_threshold = calculate_optimal_threshold(labels_test, preds_prob)
    mcc = matthews_corrcoef(labels_test, (preds_prob > optimal_threshold).astype(int))

    metrics_vector_rf.append([auc_score, pr_auc, mcc])

def train_and_evaluate_lr(data_train, labels_train, data_test, labels_test, metrics_vector_lr):
    data_train_flattened = data_train.reshape(data_train.shape[0], -1)
    data_test_flattened = data_test.reshape(data_test.shape[0], -1)

    lr_model = LogisticRegression(class_weight='balanced', max_iter=5000)
    lr_model.fit(data_train_flattened, labels_train.ravel())
    preds_prob = lr_model.predict_proba(data_test_flattened)[:, 1]
    auc_score = roc_auc_score(labels_test, preds_prob)
    precision, recall, _ = precision_recall_curve(labels_test, preds_prob)
    pr_auc = auc(recall, precision)
    optimal_threshold = calculate_optimal_threshold(labels_test, preds_prob)
    mcc = matthews_corrcoef(labels_test, (preds_prob > optimal_threshold).astype(int))

    metrics_vector_lr.append([auc_score, pr_auc, mcc])
    
def train_and_evaluate_svc(data_train, labels_train, data_test, labels_test, metrics_vector_svc):
    data_train_flattened = data_train.reshape(data_train.shape[0], -1)
    data_test_flattened = data_test.reshape(data_test.shape[0], -1)

    svc_model = LinearSVC(class_weight='balanced', max_iter=5000)
    calibrated_svc = CalibratedClassifierCV(svc_model, method='sigmoid', cv=5)
    calibrated_svc.fit(data_train_flattened, labels_train.ravel())
    preds_prob = calibrated_svc.predict_proba(data_test_flattened)[:, 1]
    auc_score = roc_auc_score(labels_test, preds_prob)
    precision, recall, _ = precision_recall_curve(labels_test, preds_prob)
    pr_auc = auc(recall, precision)
    optimal_threshold = calculate_optimal_threshold(labels_test, preds_prob)
    mcc = matthews_corrcoef(labels_test, (preds_prob > optimal_threshold).astype(int))

    metrics_vector_svc.append([auc_score, pr_auc, mcc])

def train_and_evaluate_knn(data_train, labels_train, data_test, labels_test, metrics_vector_knn):
    data_train_flattened = data_train.reshape(data_train.shape[0], -1)
    data_test_flattened = data_test.reshape(data_test.shape[0], -1)

    knn_model = KNeighborsClassifier(n_neighbors=5)
    knn_model.fit(data_train_flattened, labels_train.ravel())
    preds_prob = knn_model.predict_proba(data_test_flattened)[:, 1]
    auc_score = roc_auc_score(labels_test, preds_prob)
    precision, recall, _ = precision_recall_curve(labels_test, preds_prob)
    pr_auc = auc(recall, precision)
    optimal_threshold = calculate_optimal_threshold(labels_test, preds_prob)
    mcc = matthews_corrcoef(labels_test, (preds_prob > optimal_threshold).astype(int))

    metrics_vector_knn.append([auc_score, pr_auc, mcc])

def train_and_evaluate_model(model, data_train, labels_train, data_test, labels_test, metrics_vector):
    smote = SMOTE(sampling_strategy='auto')
    under_sampler = RandomUnderSampler(sampling_strategy='auto')
    
    resampling_pipeline = imbalanced_pipeline([
        ('smote', smote),
        ('under', under_sampler)
    ])
    
    data_train_resampled, labels_train_resampled = resampling_pipeline.fit_resample(data_train.reshape(data_train.shape[0], -1), labels_train.ravel())
    data_train_resampled = data_train_resampled.reshape(data_train_resampled.shape[0], pNum, 2)
    
    dnn.train_DNN(model, data_train_resampled, labels_train_resampled, callback, class_weights=class_weights)
    yhat_test_prob = model.predict(data_test)

    fpr, tpr, thresholds = roc_curve(labels_test, yhat_test_prob)
    roc_auc = auc(fpr, tpr)
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]
    yhat_test = (yhat_test_prob >= optimal_threshold).astype(int)

    true_values = pd.Series(labels_test.ravel())
    predicted_values = pd.Series(yhat_test.ravel())
    label_encoder = LabelEncoder()
    true_values = label_encoder.fit_transform(true_values)
    predicted_values = label_encoder.transform(predicted_values)

    cm = confusion_matrix(true_values, predicted_values)
    tn, fp, fn, tp = cm.ravel()
    accuracy = accuracy_score(true_values, predicted_values)
    sensitivity = recall_score(true_values, predicted_values)
    specificity = tn / (tn + fp)
    precision = precision_score(true_values, predicted_values)
    f1 = f1_score(true_values, predicted_values)
    mcc = matthews_corrcoef(true_values, predicted_values)
    precision_recall_auc = average_precision_score(true_values, yhat_test_prob)
    
    metrics = [accuracy, sensitivity, specificity, precision, f1, roc_auc, precision_recall_auc, mcc, i]
    metrics_vector.append(metrics)

if __name__ == "__main__":
    n_folds = 10
    data, labels, header_info = load_data()
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True)
    metrics_vector = []
    metrics_vector_xgb = []
    metrics_vector_rf = []
    metrics_vector_lr = []
    metrics_vector_svc = []
    metrics_vector_knn = []

    for i, (train_idx, test_idx) in enumerate(skf.split(data, labels)):
        print("Running Fold", i+1, "/", n_folds)
        data_train, labels_train = data[train_idx], labels[train_idx]
        data_test, labels_test = data[test_idx], labels[test_idx]

        model, callback = create_model()
        train_and_evaluate_model(model, data_train, labels_train, data_test, labels_test, metrics_vector)
        train_and_evaluate_xgboost(data_train, labels_train, data_test, labels_test, metrics_vector_xgb)
        train_and_evaluate_rf(data_train, labels_train, data_test, labels_test, metrics_vector_rf)
        train_and_evaluate_lr(data_train, labels_train, data_test, labels_test, metrics_vector_lr)
        train_and_evaluate_svc(data_train, labels_train, data_test, labels_test, metrics_vector_svc)
        train_and_evaluate_knn(data_train, labels_train, data_test, labels_test, metrics_vector_knn)

    mean_auc_xgb = np.mean([x[0] for x in metrics_vector_xgb])
    mean_pr_auc_xgb = np.mean([x[1] for x in metrics_vector_xgb])
    mean_mcc_xgb = np.mean([x[2] for x in metrics_vector_xgb])

    mean_auc_rf = np.mean([x[0] for x in metrics_vector_rf])
    mean_pr_auc_rf = np.mean([x[1] for x in metrics_vector_rf])
    mean_mcc_rf = np.mean([x[2] for x in metrics_vector_rf])

    mean_auc_lr = np.mean([x[0] for x in metrics_vector_lr])
    mean_pr_auc_lr = np.mean([x[1] for x in metrics_vector_lr])
    mean_mcc_lr = np.mean([x[2] for x in metrics_vector_lr])

    mean_auc_svc = np.mean([x[0] for x in metrics_vector_svc])
    mean_pr_auc_svc = np.mean([x[1] for x in metrics_vector_svc])
    mean_mcc_svc = np.mean([x[2] for x in metrics_vector_svc])

    mean_auc_knn = np.mean([x[0] for x in metrics_vector_knn])
    mean_pr_auc_knn = np.mean([x[1] for x in metrics_vector_knn])
    mean_mcc_knn = np.mean([x[2] for x in metrics_vector_knn])

    print(f"Mean AUC XGBoost: {mean_auc_xgb}")
    print(f"Mean prAUC XGBoost: {mean_pr_auc_xgb}")
    print(f"Mean MCC XGBoost: {mean_mcc_xgb}")

    print(f"Mean AUC Random Forest: {mean_auc_rf}")
    print(f"Mean prAUC Random Forest: {mean_pr_auc_rf}")
    print(f"Mean MCC Random Forest: {mean_mcc_rf}")

    print(f"Mean AUC Logistic Regression: {mean_auc_lr}")
    print(f"Mean prAUC Logistic Regression: {mean_pr_auc_lr}")
    print(f"Mean MCC Logistic Regression: {mean_mcc_lr}")

    print(f"Mean AUC LinearSVC: {mean_auc_svc}")
    print(f"Mean prAUC LinearSVC: {mean_pr_auc_svc}")
    print(f"Mean MCC LinearSVC: {mean_mcc_svc}")

    print(f"Mean AUC KNeighborsClassifier: {mean_auc_knn}")
    print(f"Mean prAUC KNeighborsClassifier: {mean_pr_auc_knn}")
    print(f"Mean MCC KNeighborsClassifier: {mean_mcc_knn}")

    column_names = ['Accuracy', 'Sensitivity', 'Specificity', 'Precision', 'F1 Score', 'ROC_AUC', 'prAUC', 'MCC', 'Fold']
    df = pd.DataFrame(metrics_vector, columns=column_names)

    for i in range(len(metrics_vector)):
        metrics_vector[i].extend(metrics_vector_xgb[i])
        metrics_vector[i].extend(metrics_vector_rf[i])
        metrics_vector[i].extend(metrics_vector_lr[i])
        metrics_vector[i].extend(metrics_vector_svc[i])
        metrics_vector[i].extend(metrics_vector_knn[i])

    column_names.extend(['XGBoost_AUC', 'XGBoost_prAUC', 'XGBoost_MCC', 'RandomForest_AUC', 'RandomForest_prAUC', 'RandomForest_MCC', 'LogisticRegression_AUC', 'LogisticRegression_prAUC', 'LogisticRegression_MCC', 'LinearSVC_AUC', 'LinearSVC_prAUC', 'LinearSVC_MCC', 'KNeighbors_AUC', 'KNeighbors_prAUC', 'KNeighbors_MCC'])
    df = pd.DataFrame(metrics_vector, columns=column_names)

    mean_row = df.mean(axis=0)
    mean_row['Fold'] = 'Mean'
    df = pd.concat([df, pd.DataFrame([mean_row])], ignore_index=True)

    df.to_csv(dataFolderPath + os.path.sep + paralabel + os.path.sep + "02_23_SEPSISincidence_STATknockoffs_metrics_summary_dropout005_withXGB_RF_LR_SVC_KNN.csv"); #or PRSKnockoffs
